In [2]:
from pyspark.sql import SparkSession

# Initialize a simple Spark session
spark = SparkSession.builder \
    .appName("Simple Test") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Create a simple DataFrame
data = [("Alice", 1), ("Bob", 2)]
df = spark.createDataFrame(data, ["Name", "Id"])


# Show the DataFrame
df.show()

# Stop the Spark session
spark.stop()


+-----+---+
| Name| Id|
+-----+---+
|Alice|  1|
|  Bob|  2|
+-----+---+



In [6]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc.version)


3.5.3


In [6]:
import pyspark
import org.apache.hadoop.util.VersionInfo
from py4j.java_gateway import java_import

print(pyspark.__version__)

print(VersionInfo.getVersion())

java_import(spark._jvm, "org.apache.hadoop.util.VersionInfo")
hadoop_version = spark._jvm.VersionInfo.getVersion()
print(hadoop_version)


ModuleNotFoundError: No module named 'org'

In [3]:
from pyspark.sql import SparkSession

spark.sql("SELECT current_version()")

AnalysisException: [UNRESOLVED_ROUTINE] Cannot resolve function `current_version` on search path [`system`.`builtin`, `system`.`session`, `spark_catalog`.`default`].; line 1 pos 7

In [4]:
# Initialize SparkSession
from pyspark.sql import SparkSession

# Initialize SparkSession with Iceberg JAR
spark = SparkSession.builder \
    .appName("MinIO Connection") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog")\
    .config("spark.sql.catalog.my_catalog.type", "hadoop")\
    .config("spark.sql.catalog.my_catalog.warehouse", "s3a://iceberg-bucket/warehouse")\
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12-1.6.1.jar") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()


spark.sql("SELECT current_version()")



# spark.sql("Create Table stg.product(productID INT, productName string) STORED AS ORC")

# spark.sql("DROP Table stg.product")
spark.sql("Show Databases").show()
spark.sql("Show Schemas").show()
spark.sql("Show catalogs").show()



# spark.sql("""
# CREATE TABLE stg.product (
#     productID INT,
#     productName STRING
# ) 
# USING iceberg 
# TBLPROPERTIES (
#     'format-version' = '2',  -- Use Iceberg format version 2
#     'write.format.default' = 'orc'  -- Set default write format to ORC
# )
# """)


AnalysisException: [UNRESOLVED_ROUTINE] Cannot resolve function `current_version` on search path [`system`.`builtin`, `system`.`session`, `spark_catalog`.`default`].; line 1 pos 7

In [5]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .appName("IcebergCatalogSetup")
    .config("spark.sql.catalog.staging", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.staging.catalog-impl", "org.apache.iceberg.hadoop.HadoopCatalog")
    .config("spark.sql.catalog.staging.warehouse", "s3a://iceberg-bucket/warehouse")
    .config("spark.sql.catalog.staging.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.sql.catalog.staging.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.sql.catalog.staging.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.sql.catalog.staging.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
    .config("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")
    .getOrCreate())

# spark.sql("SHOW CATALOGS").show()
# spark.sql("SHOW DATABASES").show()
# spark.sql("SHOW SCHEMAS").show()

# Create the database (namespace) called `stage` and schema `stg`
# spark.sql("CREATE NAMESPACE IF NOT EXISTS staging.stage.stg")

# Create the table `product` with the specified columns
spark.sql("create TABLE IF NOT EXISTS staging.stage.stg.product (ProductID STRING, ProductName STRING, SupplierID STRING, CategoryID STRING, QuantityPerUnit STRING, UnitPrice STRING, UnitsInStock string, UnitsOnOrder string, ReorderLevel string, Discontinued string)")


Py4JJavaError: An error occurred while calling o103.sql.
: java.lang.NoSuchMethodError: org.apache.hadoop.util.SemaphoredDelegatingExecutor.<init>(Lcom/google/common/util/concurrent/ListeningExecutorService;IZ)V
	at org.apache.hadoop.fs.s3a.S3AFileSystem.create(S3AFileSystem.java:772)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.iceberg.hadoop.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.iceberg.TableMetadataParser.internalWrite(TableMetadataParser.java:125)
	at org.apache.iceberg.TableMetadataParser.write(TableMetadataParser.java:119)
	at org.apache.iceberg.hadoop.HadoopTableOperations.commit(HadoopTableOperations.java:157)
	at org.apache.iceberg.BaseMetastoreCatalog$BaseMetastoreCatalogTableBuilder.create(BaseMetastoreCatalog.java:201)
	at org.apache.iceberg.CachingCatalog$CachingTableBuilder

In [9]:
spark.sql("CREATE TABLE my_catalog.db_name.my_table (id INT, name STRING) USING iceberg")

Py4JJavaError: An error occurred while calling o97.sql.
: java.lang.NoSuchMethodError: org.apache.hadoop.util.SemaphoredDelegatingExecutor.<init>(Lcom/google/common/util/concurrent/ListeningExecutorService;IZ)V
	at org.apache.hadoop.fs.s3a.S3AFileSystem.create(S3AFileSystem.java:772)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.iceberg.hadoop.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.iceberg.TableMetadataParser.internalWrite(TableMetadataParser.java:125)
	at org.apache.iceberg.TableMetadataParser.write(TableMetadataParser.java:119)
	at org.apache.iceberg.hadoop.HadoopTableOperations.commit(HadoopTableOperations.java:157)
	at org.apache.iceberg.BaseMetastoreCatalog$BaseMetastoreCatalogTableBuilder.create(BaseMetastoreCatalog.java:201)
	at org.apache.iceberg.CachingCatalog$CachingTableBuilder.

In [56]:
# Drop the existing table
spark.sql("DROP TABLE IF EXISTS stg.product")

# Create a new Iceberg table with v2 properties
spark.sql("""
CREATE TABLE stg.product (
    productID INT,
    productName STRING
)
USING iceberg
TBLPROPERTIES (
    'format-version' = '2',  
    'write.format.default' = 'orc'
)
""")


DataFrame[]

In [58]:
df.write \
    .format("iceberg") \
    .mode("append") \
    .save("spark_catalog.stg.product")


Py4JJavaError: An error occurred while calling o634.save.
: org.apache.iceberg.hive.RuntimeMetaException: Failed to connect to Hive Metastore
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:85)
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:34)
	at org.apache.iceberg.ClientPoolImpl.get(ClientPoolImpl.java:143)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:70)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:65)
	at org.apache.iceberg.hive.CachedClientPool.run(CachedClientPool.java:122)
	at org.apache.iceberg.hive.HiveTableOperations.doRefresh(HiveTableOperations.java:147)
	at org.apache.iceberg.BaseMetastoreTableOperations.refresh(BaseMetastoreTableOperations.java:90)
	at org.apache.iceberg.BaseMetastoreTableOperations.current(BaseMetastoreTableOperations.java:73)
	at org.apache.iceberg.BaseMetastoreCatalog.loadTable(BaseMetastoreCatalog.java:49)
	at org.apache.iceberg.spark.SparkCatalog.load(Spa

In [55]:
# Sample data to insert
data = [(1, "Product A"), (2, "Product B"), (3, "Product C")]

# Create a DataFrame from the sample data
df = spark.createDataFrame(data, ["productID", "productName"])

# Write the DataFrame to the Iceberg table
df.writeTo("stg.product").append()

AnalysisException: Cannot write into v1 table: `spark_catalog`.`stg`.`product`.

In [47]:
# Drop the existing table
spark.sql("DROP TABLE IF EXISTS stg.product")

# Create a new Iceberg table with v2 properties
spark.sql("""
CREATE TABLE stg.product (
    productID INT,
    productName STRING
)
USING iceberg
TBLPROPERTIES (
    'format-version' = '2',  
    'write.format.default' = 'orc'
)
""")


DataFrame[]

In [49]:
# Sample data to insert
data = [(1, "Product A"), (2, "Product B"), (3, "Product C")]

# Create a DataFrame from the sample data
df = spark.createDataFrame(data, ["productID", "productName"])

# Write the DataFrame to the Iceberg table using the 'format' method
df.write \
    .format("iceberg") \
    .mode("append") \
    .save("stg.product")


Py4JJavaError: An error occurred while calling o565.save.
: org.apache.iceberg.hive.RuntimeMetaException: Failed to connect to Hive Metastore
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:85)
	at org.apache.iceberg.hive.HiveClientPool.newClient(HiveClientPool.java:34)
	at org.apache.iceberg.ClientPoolImpl.get(ClientPoolImpl.java:143)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:70)
	at org.apache.iceberg.ClientPoolImpl.run(ClientPoolImpl.java:65)
	at org.apache.iceberg.hive.CachedClientPool.run(CachedClientPool.java:122)
	at org.apache.iceberg.hive.HiveTableOperations.doRefresh(HiveTableOperations.java:147)
	at org.apache.iceberg.BaseMetastoreTableOperations.refresh(BaseMetastoreTableOperations.java:90)
	at org.apache.iceberg.BaseMetastoreTableOperations.current(BaseMetastoreTableOperations.java:73)
	at org.apache.iceberg.BaseMetastoreCatalog.loadTable(BaseMetastoreCatalog.java:49)
	at org.apache.iceberg.spark.SparkCatalog.load(Spa

In [52]:
spark.sql("DESCRIBE FORMATTED stg.product").show(truncate=False)

+----------------------------+---------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                  |comment|
+----------------------------+---------------------------------------------------------------------------+-------+
|productID                   |int                                                                        |NULL   |
|productName                 |string                                                                     |NULL   |
|                            |                                                                           |       |
|# Detailed Table Information|                                                                           |       |
|Catalog                     |spark_catalog                                                              |       |
|Database                    |stg                                               

In [28]:
spark.sql("SHOW TABLES IN stg").show()
# Sample data to insert
data = [(1, "Product A"), (2, "Product B"), (3, "Product C")]

# Create a DataFrame from the sample data
df = spark.createDataFrame(data, ["productID", "productName"])

# Write the DataFrame to the Iceberg table
df.writeTo("stg.product").append()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|      stg|  product|      false|
+---------+---------+-----------+



AnalysisException: Cannot write into v1 table: `spark_catalog`.`stg`.`product`.

In [26]:
spark.sql("Select * From stg.product")

Py4JJavaError: An error occurred while calling o93.sql.
: java.util.concurrent.ExecutionException: org.apache.spark.sql.AnalysisException: iceberg is not a valid Spark SQL Data Source.
	at org.sparkproject.guava.util.concurrent.AbstractFuture$Sync.getValue(AbstractFuture.java:306)
	at org.sparkproject.guava.util.concurrent.AbstractFuture$Sync.get(AbstractFuture.java:293)
	at org.sparkproject.guava.util.concurrent.AbstractFuture.get(AbstractFuture.java:116)
	at org.sparkproject.guava.util.concurrent.Uninterruptibles.getUninterruptibly(Uninterruptibles.java:135)
	at org.sparkproject.guava.cache.LocalCache$Segment.getAndRecordStats(LocalCache.java:2410)
	at org.sparkproject.guava.cache.LocalCache$Segment.loadSync(LocalCache.java:2380)
	at org.sparkproject.guava.cache.LocalCache$Segment.lockedGetOrLoad(LocalCache.java:2342)
	at org.sparkproject.guava.cache.LocalCache$Segment.get(LocalCache.java:2257)
	at org.sparkproject.guava.cache.LocalCache.get(LocalCache.java:4000)
	at org.sp

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Iceberg") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1") \
    .getOrCreate()

# spark.sql("show databases").show()
# spark.sql("Create Database stage")
# spark.sql("show catalogs").show()
# spark.sql("Create namespace if not exists stg")
# spark.sql("describe catalog spark_catalog")
catalog_properties = {
    "type": "hive",  # or "hive", "glue", etc.
    "warehouse": "hdfs:///iceberg/warehouse"  # Update this path
}

spark.sql(r"""
create catalog stg
          """)

# Register the catalog with Spark
spark.sql(r"""create catalog my_catalog
    with (
        'type' = '{catalog_properties["type"]}',
        'warehouse' = '{catalog_properties["warehouse"]}'
    )
""")


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'catalog'.(line 2, pos 7)

== SQL ==

create catalog stg
-------^^^
          


In [25]:
# Create an Iceberg table
# Create a new catalog
catalog_properties = {
    "type": "hadoop",  # or "hive", "glue", etc.
    "warehouse": "hdfs:///iceberg/warehouse"  # Update this path
}

spark.sql(r"""
create catalog stg
          """)
# Register the catalog with Spark
spark.sql(r"""create catalog my_catalog
    with (
        'type' = '{catalog_properties["type"]}',
        'warehouse' = '{catalog_properties["warehouse"]}'
    )
""")

# Show catalogs to verify
spark.sql("show catalogs").show()


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'catalog'.(line 2, pos 7)

== SQL ==

create catalog stg
-------^^^
          


In [10]:
pip uninstall pyspark
pip install pyspark


SyntaxError: invalid syntax (2298578304.py, line 1)

In [7]:
import pandas as pd

# Define the MinIO bucket information
minio_bucket = "basebucket"  # Your bucket name
minio_file = "northwind/products.csv"  # Path to your file in the bucket
minio_url = f"s3://{minio_bucket}/{minio_file}"

# MinIO credentials
access_key = "minioadmin"
secret_key = "minioadmin"

# Read the CSV file using pandas with s3fs
try:
    df = pd.read_csv(
        minio_url,
        storage_options={
            'key': access_key,
            'secret': secret_key,
            'client_kwargs': {
                'endpoint_url': 'http://localhost:9000',  # Correct endpoint for API port
                'use_ssl': False  # Set to True if using HTTPS
            }
        }
    )
    # Display the DataFrame
    
except Exception as e:
    print("Error reading CSV:", e)


In [11]:
# import the pandas
import pandas as pd

# from pyspark library import 
# SparkSession
from pyspark.sql import SparkSession

# Building the SparkSession and name
# it :'pandas to spark'
spark = SparkSession.builder.appName(
"pandas to spark").getOrCreate()

# Create the DataFrame with the help 
# of pd.DataFrame()

# create DataFrame
df_spark = spark.createDataFrame(df)

df_spark.show(10)


+---------+--------------------+----------+----------+-------------------+---------+------------+------------+------------+------------+
|ProductID|         ProductName|SupplierID|CategoryID|    QuantityPerUnit|UnitPrice|UnitsInStock|UnitsOnOrder|ReorderLevel|Discontinued|
+---------+--------------------+----------+----------+-------------------+---------+------------+------------+------------+------------+
|        1|                Chai|         1|         1| 10 boxes x 20 bags|     18.0|          39|           0|          10|           0|
|        2|               Chang|         1|         1| 24 - 12 oz bottles|     19.0|          17|          40|          25|           0|
|        3|       Aniseed Syrup|         1|         2|12 - 550 ml bottles|     10.0|          13|          70|          25|           0|
|        4|Chef Anton's Caju...|         2|         2|     48 - 6 oz jars|     22.0|          53|           0|           0|           0|
|        5|Chef Anton's Gumb...|         

In [4]:
from pyfpe_ff3 import FF3Cipher

key = "EF4359D8D580AA4F7F036D6F04FC6A94"
tweak = "D8E7920AFA330A73"
c = FF3Cipher(key, tweak)

plaintext = "4000001234567899"
ciphertext = c.encrypt(plaintext)
decrypted = c.decrypt(ciphertext)

print(f"{plaintext} -> {ciphertext} -> {decrypted}")

# format encrypted value
ccn = f"{ciphertext[:4]} {ciphertext[4:8]} {ciphertext[8:12]} {ciphertext[12:]}"
print(f"Encrypted CCN value with formatting: {ccn}")

4000001234567899 -> 5107085540698446 -> 4000001234567899
Encrypted CCN value with formatting: 5107 0855 4069 8446
